In [40]:
import numpy as np
import os
import scipy
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [84]:
glove_file = datapath(os.path.abspath('data/glove.6B.300d.txt'))
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

C:\Users\Owner\AppData\Local\Temp\ipykernel_8996\3660045629.py:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, tmp_file)


In [101]:
model.sort_by_descending_frequency()

In [73]:
# model.distances("king", ["car", "queen", "man", "jack", "woman"])
print(model.similarity("king", "queen"))
print(model.similarity("king", "jack"))
print(model.similarity("king", "car"))
print(model.similarity("king", "fish"))
print(model.similarity("king", "turtle"))
print(model.similarity("fish", "turtle"))
print(model.similarity("goldfish", "koi"))
print(model.similarity("king", "crown"))
print(model.similarity("king", "throne"))

0.6336469
0.2231067
0.12324726
0.16535372
0.063374296
0.4094217
0.4485258
0.52248627
0.5606488


In [77]:
model.most_similar("fish")

[('salmon', 0.6541995406150818),
 ('tuna', 0.6304336786270142),
 ('shrimp', 0.6116418242454529),
 ('meat', 0.6113753914833069),
 ('seafood', 0.605174720287323),
 ('fishes', 0.604816734790802),
 ('trout', 0.5963797569274902),
 ('eat', 0.5483304262161255),
 ('fishing', 0.5461738705635071),
 ('shark', 0.5428288578987122)]

In [22]:
def word_similarity(word1: str, word2: str, encoding_model) -> float:
    distance = scipy.spatial.distance.euclidean(encoding_model[word1], encoding_model[word2])
    return distance

In [33]:
word1 = "toyota"
word2 = "honda"

word_similarity(word1, word2, model)

5.509447667801142

In [79]:
model.most_similar("corolla")

[('camry', 0.634617805480957),
 ('rav4', 0.5531828999519348),
 ('corollas', 0.5511930584907532),
 ('toyota', 0.5509843230247498),
 ('hatchback', 0.550690233707428),
 ('sedan', 0.5453417897224426),
 ('yaris', 0.5359854698181152),
 ('prius', 0.5339162349700928),
 ('sedans', 0.5198630094528198),
 ('tercel', 0.5056036114692688)]

In [100]:
model.most_similar(model.index_to_key[42000])

[('peake', 0.48120614886283875),
 ('wallabies', 0.39346224069595337),
 ('croom', 0.3829837143421173),
 ('dervla', 0.38141125440597534),
 ('charlesworth', 0.3736904263496399),
 ('wallaby', 0.37211546301841736),
 ('alcock', 0.3676917254924774),
 ('mceuen', 0.3651144206523895),
 ('lomu', 0.3630010485649109),
 ('bilic', 0.361842542886734)]

In [112]:
i = 5000
model.index_to_key[i:i+15]

['technique',
 'inspection',
 'situations',
 'symptoms',
 'risen',
 'properly',
 'ferry',
 'folk',
 'foods',
 'derby',
 '1951',
 'achievement',
 'demonstrated',
 'preserve',
 'dropping']

In [120]:
model.most_similar(positive=["king"])

[('queen', 0.6336469054222107),
 ('prince', 0.6196622848510742),
 ('monarch', 0.5899620652198792),
 ('kingdom', 0.5791266560554504),
 ('throne', 0.5606487989425659),
 ('ii', 0.5562329292297363),
 ('iii', 0.5503199100494385),
 ('crown', 0.5224862694740295),
 ('reign', 0.5217353701591492),
 ('kings', 0.5066401958465576)]

In [150]:
def generate_category(starting_word: str, model):
    words = [starting_word]
    for _ in range(3):
        words.append(model.most_similar(positive=words, restrict_vocab=10_000)[0])
    return words

In [159]:
generate_category("pair", model)

['pair',
 ('pairs', 0.7135139107704163),
 ('two', 0.5374454259872437),
 ('three', 0.6627466082572937)]